# Analysis Code for 2-Point Correlator

## Preparation

In [2]:
%pip install numpy scipy iminuit matplotlib scienceplots
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker
import scienceplots

Note: you may need to restart the kernel to use updated packages.


## Potentials